In [ ]:
# To mount our Google Drive folder if we are using Google Colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Installation Steps

In [ ]:
#Install tqdm for progress bars
!pip install --quiet tqdm

#Install pandas
!pip install --quiet pandas

#Install sklearn
!pip install --quiet sklearn

#Install termcolor
!pip install --quiet termcolor

# Verify Installations

In [ ]:
!sudo apt install python3

# Data Preprocessing

## Fetch data from filtered csvs

In [ ]:
import pandas as pd
import numpy as np

from termcolor import colored
from tqdm import tqdm

# testPath = "./filtered_data/test.csv"

# For Google Colab
testPath = "./drive/MyDrive/preprocessed_data/test.csv"
dfTest = pd.read_csv(testPath)

dfTest.head()

,claim,label,annotated-questions,label-binary
0,"Says Kentucky Rep. Andy Barr ""would let shady ...",half-true,"1. Has Barr received $36,550 from payday lende...",1
1,"""New reports show Kevin Nicholson made over $1...",barely-true,1. Did Nicholson make $1 million consulting fo...,0
2,Says that unless the recipient called back abo...,pants-fire,1. Will people be taken into local police cust...,0
3,"""Donald Trump said he was excited for the 2008...",half-true,"1. Was Donald Trump ""Excited"" for the 2008 hou...",1
4,"""The president has said the national security ...",barely-true,1. Has Obama cited climate change as the top n...,0


## Optional code to verify dataframe

In [ ]:
#Questions are of type str
print(type(dfTest.loc[0, 'annotated-questions']))
dfTest.loc[0, 'annotated-questions']

<class 'str'>


"1. Has Barr received $36,550 from payday lenders? \n2. Did Barr vote for legislation that would weaken restrictions for payday lenders? \n3. Are there any protections for service members using payday lending services? \n4. Has Barr's voting record directly affected protection for veterans against payday lenders? \n5. Did Bar accept $36,000 from pay day lenders? \n6. Does Bar let pay day lenders take advantage of troops?"

## Preprocess Question strings

In [ ]:
# Convert each question string into an array for easier evaluation
dfTest['annotated-questions'] = dfTest['annotated-questions'].str.split('\n')
dfTest.head()

,claim,label,annotated-questions,label-binary
0,"Says Kentucky Rep. Andy Barr ""would let shady ...",half-true,"[1. Has Barr received $36,550 from payday lend...",1
1,"""New reports show Kevin Nicholson made over $1...",barely-true,[1. Did Nicholson make $1 million consulting f...,0
2,Says that unless the recipient called back abo...,pants-fire,[1. Will people be taken into local police cus...,0
3,"""Donald Trump said he was excited for the 2008...",half-true,"[1. Was Donald Trump ""Excited"" for the 2008 ho...",1
4,"""The president has said the national security ...",barely-true,[1. Has Obama cited climate change as the top ...,0


## Evaluate Claim Decomp Performance

In [ ]:
# Create default cols to store quin results
dfTest['decomp-subquestions'] = ""
dfTest['decomp-subclaims'] = ""
dfTest['decomp-supporting'] = 0
dfTest['decomp-refuting'] = 0
dfTest['decomp-label'] = ""
dfTest['decomp-label-binary'] = 0
dfTest['decomp-evidence'] = ""
dfTest['decomp-performance'] = 0

dfTest.head()

,claim,label,annotated-questions,label-binary,decomp-subquestions,decomp-subclaims,decomp-supporting,decomp-refuting,decomp-label,decomp-label-binary,decomp-evidence,decomp-performance
0,"Says Kentucky Rep. Andy Barr ""would let shady ...",half-true,"[1. Has Barr received $36,550 from payday lend...",1,,,0,0,,0,,0
1,"""New reports show Kevin Nicholson made over $1...",barely-true,[1. Did Nicholson make $1 million consulting f...,0,,,0,0,,0,,0
2,Says that unless the recipient called back abo...,pants-fire,[1. Will people be taken into local police cus...,0,,,0,0,,0,,0
3,"""Donald Trump said he was excited for the 2008...",half-true,"[1. Was Donald Trump ""Excited"" for the 2008 ho...",1,,,0,0,,0,,0
4,"""The president has said the national security ...",barely-true,[1. Has Obama cited climate change as the top ...,0,,,0,0,,0,,0


In [ ]:
!pip install aiohttp
!pip install nest-asyncio
!pip install BeautifulSoup

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 26.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for 

## Decompose original claims

### Step 1
Here, we will call a deployed version of our claim decomposition model to decompose the given claims into binary sub-questions.

It relies on the following backend API: https://adv-decomp-hnozmogerq-as.a.run.app/advdecomp


In [ ]:
import asyncio
import aiohttp
import nest_asyncio
import json

nest_asyncio.apply()

# define the base URL and parameters
decomp_url = 'https://adv-decomp-hnozmogerq-as.a.run.app/advdecomp'

async def decomp(url, claim):
  params = {'claim': claim}
  async with aiohttp.ClientSession() as session:
        async with session.get(url, params=params) as response:
            content = await response.text(encoding='unicode_escape')
            return content

async def processDecomp(df):
  for i in tqdm(range(len(df))):
    claim = df.loc[i, 'claim']
    response = await decomp(decomp_url, claim)
    response = response.strip()
    # Get the array of sub-questions
    df.loc[i, 'decomp-subquestions'] = response

# print the response content
asyncio.run(processDecomp(dfTest))

100%|██████████| 200/200 [09:03<00:00,  2.72s/it]


In [ ]:
dfTest.dtypes

In [ ]:
dfTest.head()

,claim,label,annotated-questions,label-binary,decomp-subquestions,decomp-subclaims,decomp-supporting,decomp-refuting,decomp-label,decomp-label-binary,decomp-evidence,decomp-performance
0,"Says Kentucky Rep. Andy Barr ""would let shady ...",half-true,"[1. Has Barr received $36,550 from payday lend...",1,""" 1. Did Barr take money from these lenders?\n...",,0,0,,0,,0
1,"""New reports show Kevin Nicholson made over $1...",barely-true,[1. Did Nicholson make $1 million consulting f...,0,""" 1. Did Nicholson make over $1 million?\n2. D...",,0,0,,0,,0
2,Says that unless the recipient called back abo...,pants-fire,[1. Will people be taken into local police cus...,0,""" 1. Did someone call about this person's tax ...",,0,0,,0,,0
3,"""Donald Trump said he was excited for the 2008...",half-true,"[1. Was Donald Trump ""Excited"" for the 2008 ho...",1,""" 1. Did Donald Trump say he was excited for t...",,0,0,,0,,0
4,"""The president has said the national security ...",barely-true,[1. Has Obama cited climate change as the top ...,0,""" 1. Did the president say that global warming...",,0,0,,0,,0


In [ ]:
dfTest.to_csv("./result/evalDecompTest-stage1.csv", index=False, encoding = 'utf-8-sig', header=True, escapechar='',)

## Transform sub-questions into sub-claims
### Step 2
Now that we have our decomposed binary questions, we will transform them into a hypothesis using another deployed model.

It relies on the following backend API for question-hypothesis conversion: https://hypothesize-hnozmogerq-as.a.run.app/hypothesize



In [ ]:
# define the base URL and parameters
hypo_url = 'https://hypothesize-hnozmogerq-as.a.run.app/hypothesize'

async def hypothesize(url, q):
  params = {'question': q}
  async with aiohttp.ClientSession() as session:
        async with session.get(url, params=params) as response:
            content = await response.text(encoding='unicode_escape')
            return content

async def processHypo(df):
  for i in tqdm(range(len(df))):
    subQ = dfTest.loc[i, 'decomp-subquestions'].split("\n")
    subQ = [str[str.index(".") + 2:] if "." in str else str for str in subQ]
    hypothesisList = ""
    for j in tqdm(range(len(subQ))):
      response = await hypothesize(hypo_url, subQ[j])
      response = response.strip()
      hypothesisList = hypothesisList + response + ";"
    
    df.loc[i, 'decomp-subclaims'] = hypothesisList[:-1]

# print the response content
asyncio.run(processHypo(dfTest))

100%|██████████| 200/200 [13:12<00:00,  3.96s/it]


In [ ]:
dfTest.head()

,claim,label,annotated-questions,label-binary,decomp-subquestions,decomp-subclaims,decomp-supporting,decomp-refuting,decomp-label,decomp-label-binary,decomp-evidence,decomp-performance
0,"Says Kentucky Rep. Andy Barr ""would let shady ...",half-true,"[1. Has Barr received $36,550 from payday lend...",1,""" 1. Did Barr take money from these lenders?\n...","""Barr took money from these lenders."";""Barr wo...",0,0,,0,,0
1,"""New reports show Kevin Nicholson made over $1...",barely-true,[1. Did Nicholson make $1 million consulting f...,0,""" 1. Did Nicholson make over $1 million?\n2. D...","""Nicholson made over $1 million."";""Nicholson a...",0,0,,0,,0
2,Says that unless the recipient called back abo...,pants-fire,[1. Will people be taken into local police cus...,0,""" 1. Did someone call about this person's tax ...","""Someone called about this person's tax filing...",0,0,,0,,0
3,"""Donald Trump said he was excited for the 2008...",half-true,"[1. Was Donald Trump ""Excited"" for the 2008 ho...",1,""" 1. Did Donald Trump say he was excited for t...","""\nDonald Trump said he was excited for the 20...",0,0,,0,,0
4,"""The president has said the national security ...",barely-true,[1. Has Obama cited climate change as the top ...,0,""" 1. Did the president say that global warming...","""The president said that global warming is the...",0,0,,0,,0


In [ ]:
dfTest.to_csv("./result/evalDecompTest-stage2.csv", index=False, encoding = 'utf-8-sig', header=True, escapechar='',)

In [ ]:
dfTest.loc[2, 'decomp-subclaims'].replace("\";\"", ";")

'"Someone called about this person\'s tax filings.;Local cops will take this person into custody.;There were serious allegations pressed on this person\'s name.;The recipient will be taken into custody by the local cops unless they call back about "your tax filings" due to more serious allegations pressed on their name."'

## Evidence retrieval

We will now retrieve relevant evidence and carry out the entailment classification using the Quin system

#### Method 1 [Full Pipeline V1]

For our first method, we retrieve relevant articles using the original claim. Thereafter, we will conduct the entailment classification using the subclaims generated in the previous step to generate the final veracity results.

In [ ]:
newpath = "./result/evalDecompTest-stage2.csv"
dfTest = pd.read_csv(newpath)

In [ ]:
# Add extra cols for computation
dfTest['decomp-total-support'] = 0
dfTest['decomp-total-refute'] = 0
dfTest.head()

,claim,label,annotated-questions,label-binary,decomp-subquestions,decomp-subclaims,decomp-supporting,decomp-refuting,decomp-label,decomp-label-binary,decomp-evidence,decomp-performance,decomp-total-support,decomp-total-refute
0,"Says Kentucky Rep. Andy Barr ""would let shady ...",half-true,"['1. Has Barr received $36,550 from payday len...",1,""" 1. Did Barr take money from these lenders?\n...","""Barr took money from these lenders."";""Barr wo...",0;2;0;,0;0;0;,Not enough evidence;Not enough evidence;Not en...,0,"; Says Kentucky Rep. Andy Barr ""would let shad...",0,0,0
1,"""New reports show Kevin Nicholson made over $1...",barely-true,['1. Did Nicholson make $1 million consulting ...,0,""" 1. Did Nicholson make over $1 million?\n2. D...","""Nicholson made over $1 million."";""Nicholson a...",0;3;0;3;,0;0;3;1;,Not enough evidence;Probably True;Probably Fal...,0,";<b> ""New reports show Kevin Nicholson made ov...",0,0,0
2,Says that unless the recipient called back abo...,pants-fire,['1. Will people be taken into local police cu...,0,""" 1. Did someone call about this person's tax ...","""Someone called about this person's tax filing...",0;2;2;2;,0;0;0;0;,Not enough evidence;Not enough evidence;Not en...,0,Regulations issued pursuant to this type of au...,0,0,0
3,"""Donald Trump said he was excited for the 2008...",half-true,"['1. Was Donald Trump ""Excited"" for the 2008 h...",1,""" 1. Did Donald Trump say he was excited for t...","""\nDonald Trump said he was excited for the 20...",3;3;0;,0;0;3;,Probably True;Probably True;Probably False;,0,look at his history. <b>donald trump said he ...,0,0,0
4,"""The president has said the national security ...",barely-true,['1. Has Obama cited climate change as the top...,0,""" 1. Did the president say that global warming...","""The president said that global warming is the...",0;0;0;,0;0;0;,Not enough evidence;Not enough evidence;Not en...,0,;;;,0,0,0


This step relies on the following hosted Quin backend API: https://empty-meadow-488.fly.dev/api2


In [ ]:
import asyncio
import aiohttp
import nest_asyncio
import json

nest_asyncio.apply()

# define the base URL and parameters
quin_url = 'https://empty-meadow-488.fly.dev/api2'

async def fetchResutls(url, claim, subq):
  params = {'query': claim,
            'subqueries': subq}
  async with aiohttp.ClientSession() as session:
        async with session.get(url, params=params) as response:
            content = await response.text(encoding='unicode_escape')
            return content

async def evaluate(df):
  for i in tqdm(range(len(df))):
    hypothesis = df.loc[i, 'decomp-subclaims']
    hypothesis = hypothesis.replace("\";\"", ";")
    claim = df.loc[i, 'claim']
    response = await fetchResutls(quin_url, claim, hypothesis)
    response = json.loads(response)
    respArr = response['data']
    
    totalSupporting = 0
    totalRefuting = 0
    numSupporting = []
    numRefuting = []
    numSubQLabel = []
    allEvi = []
    
    if i in [111,163]:
      continue

    for j in range(len(respArr)):
      if respArr[j]['type'] == "statement":
        totalSupporting += respArr[j]['supporting']
        totalRefuting += respArr[j]['refuting']
        
        numSupporting.append(respArr[j]['supporting'])
        numRefuting.append(respArr[j]['refuting'])
        numSubQLabel.append(respArr[j]['veracity_rating'])
        
        evidence = respArr[j]['results']
        evidenceStore = ""
        for k in range(len(evidence)):
          evidenceStore += evidence[k]['snippet']
        allEvi.append(evidenceStore)
      else:
        numSupporting.append(0)
        numRefuting.append(0)
        numSubQLabel.append("Unable to evaluate")
        evidence = respArr[j]['results']
        evidenceStore = ""
        for k in range(len(evidence)):
          evidenceStore += evidence[k]['snippet']
        allEvi.append(evidenceStore)
    
    numSupporting = list(map(str, numSupporting))
    numRefuting = list(map(str, numRefuting))

    tmp = ""
    for val in numSupporting:
      tmp += val + ";"
    numSupporting = tmp

    tmp2 = ""
    for val in numRefuting:
      tmp2 += val + ";"
    numRefuting = tmp2

    tmp3 = ""
    for val in numSubQLabel:
      tmp3 += val + ";"
    numSubQLabel = tmp3

    tmp4 = ""
    for val in allEvi:
      tmp4 += val + ";"
    allEvi = tmp4

    print(numSupporting)
    print(totalSupporting)
    print(numRefuting)
    print(totalRefuting)

    df.loc[i, 'decomp-total-support'] = totalSupporting
    df.loc[i, 'decomp-total-refute'] = totalRefuting
    df.loc[i, 'decomp-supporting'] = numSupporting
    df.loc[i, 'decomp-refuting'] = numRefuting
    df.loc[i, 'decomp-label'] = numSubQLabel
    df.loc[i, 'decomp-evidence'] = allEvi

# print the response content
asyncio.run(evaluate(dfTest))

  0%|          | 1/200 [00:01<03:22,  1.02s/it]

0;2;0;
2
0;0;0;
0


  1%|          | 2/200 [00:01<03:05,  1.07it/s]

0;3;0;3;
6
0;0;3;1;
4


  2%|▏         | 3/200 [00:02<02:25,  1.35it/s]

...

2;0;1;
3
0;2;0;
2


100%|█████████▉| 199/200 [01:26<00:00,  2.87it/s]

0;0;0;
0
0;0;0;
0


100%|██████████| 200/200 [01:26<00:00,  2.31it/s]

1;1;0;
2
0;0;0;
0


In [ ]:
dfTest.head()

,claim,label,annotated-questions,label-binary,decomp-subquestions,decomp-subclaims,decomp-supporting,decomp-refuting,decomp-label,decomp-label-binary,decomp-evidence,decomp-performance,decomp-total-support,decomp-total-refute
0,"Says Kentucky Rep. Andy Barr ""would let shady ...",half-true,"['1. Has Barr received $36,550 from payday len...",1,""" 1. Did Barr take money from these lenders?\n...","""Barr took money from these lenders."";""Barr wo...",0;2;0;,0;0;0;,Not enough evidence;Not enough evidence;Not en...,0,"; Says Kentucky Rep. Andy Barr ""would let shad...",0,2,0
1,"""New reports show Kevin Nicholson made over $1...",barely-true,['1. Did Nicholson make $1 million consulting ...,0,""" 1. Did Nicholson make over $1 million?\n2. D...","""Nicholson made over $1 million."";""Nicholson a...",0;3;0;3;,0;0;3;1;,Not enough evidence;Probably True;Probably Fal...,0,";<b> ""New reports show Kevin Nicholson made ov...",0,6,4
2,Says that unless the recipient called back abo...,pants-fire,['1. Will people be taken into local police cu...,0,""" 1. Did someone call about this person's tax ...","""Someone called about this person's tax filing...",0;2;2;2;,0;0;0;0;,Not enough evidence;Not enough evidence;Not en...,0,Regulations issued pursuant to this type of au...,0,6,0
3,"""Donald Trump said he was excited for the 2008...",half-true,"['1. Was Donald Trump ""Excited"" for the 2008 h...",1,""" 1. Did Donald Trump say he was excited for t...","""\nDonald Trump said he was excited for the 20...",3;3;0;,0;0;3;,Probably True;Probably True;Probably False;,0,look at his history. <b>donald trump said he ...,0,6,3
4,"""The president has said the national security ...",barely-true,['1. Has Obama cited climate change as the top...,0,""" 1. Did the president say that global warming...","""The president said that global warming is the...",0;0;0;,0;0;0;,Not enough evidence;Not enough evidence;Not en...,0,;;;,0,0,0


In [ ]:
dfTest.to_csv("./result/evalDecompTest-stage3.csv", index=False, encoding = 'utf-8-sig', header=True, escapechar='',)

#### Method 2 [Full Pipeline V2]

For our second method, we retrieve relevant articles and conduct the entailment classification solely with the subclaims generated in the previous step to generate the final veracity results.

Cons: 

Sometimes the subclaims generated may not include sufficient contextual information to retrieve the relevant articles

In [ ]:
newpath = "./result/evalDecompTest-stage2.csv"
dfTest = pd.read_csv(newpath)

In [ ]:
# Add extra cols for computation
dfTest['decomp-total-support'] = 0
dfTest['decomp-total-refute'] = 0
dfTest.head()

,claim,label,annotated-questions,label-binary,decomp-subquestions,decomp-subclaims,decomp-supporting,decomp-refuting,decomp-label,decomp-label-binary,decomp-evidence,decomp-performance,decomp-total-support,decomp-total-refute
0,"Says Kentucky Rep. Andy Barr ""would let shady ...",half-true,"['1. Has Barr received $36,550 from payday len...",1,""" 1. Did Barr take money from these lenders?\n...","""Barr took money from these lenders."";""Barr wo...",0,0,NaN,0,NaN,0,0,0
1,"""New reports show Kevin Nicholson made over $1...",barely-true,['1. Did Nicholson make $1 million consulting ...,0,""" 1. Did Nicholson make over $1 million?\n2. D...","""Nicholson made over $1 million."";""Nicholson a...",0,0,NaN,0,NaN,0,0,0
2,Says that unless the recipient called back abo...,pants-fire,['1. Will people be taken into local police cu...,0,""" 1. Did someone call about this person's tax ...","""Someone called about this person's tax filing...",0,0,NaN,0,NaN,0,0,0
3,"""Donald Trump said he was excited for the 2008...",half-true,"['1. Was Donald Trump ""Excited"" for the 2008 h...",1,""" 1. Did Donald Trump say he was excited for t...","""\nDonald Trump said he was excited for the 20...",0,0,NaN,0,NaN,0,0,0
4,"""The president has said the national security ...",barely-true,['1. Has Obama cited climate change as the top...,0,""" 1. Did the president say that global warming...","""The president said that global warming is the...",0,0,NaN,0,NaN,0,0,0


In [ ]:
import asyncio
import aiohttp
import nest_asyncio
import json

nest_asyncio.apply()

# define the base URL and parameters
quin_url = 'https://empty-meadow-488.fly.dev/api2'

async def fetchResutlsV2(url, claim):
  params = {'query': claim }
  async with aiohttp.ClientSession() as session:
        async with session.get(url, params=params) as response:
            content = await response.text()
            return content

async def evaluateV2(df):
  for i in tqdm(range(len(df))):
    hypothesis = df.loc[i, 'decomp-subclaims']
    hypothesis = hypothesis.split("\";\"")
    
    totalSupporting = 0
    totalRefuting = 0
    numSupporting = []
    numRefuting = []
    numSubQLabel = []
    allEvi = []
    
    if i in [111,163]:
      continue

    for j in tqdm(range(len(hypothesis))):
      claim = hypothesis[j].strip()
      if claim != "":
        response = await fetchResutlsV2(quin_url, claim)
        response = json.loads(response)
        respArr = response['data']
    
      if respArr['type'] == "statement":
        totalSupporting += respArr['supporting']
        totalRefuting += respArr['refuting']
        
        numSupporting.append(respArr['supporting'])
        numRefuting.append(respArr['refuting'])
        numSubQLabel.append(respArr['veracity_rating'])
        
        evidence = respArr['results']
        evidenceStore = ""
        for k in range(len(evidence)):
          evidenceStore += evidence[k]['snippet']
        allEvi.append(evidenceStore)
      else:
        numSupporting.append(0)
        numRefuting.append(0)
        numSubQLabel.append("Unable to evaluate")
        evidence = respArr['results']
        evidenceStore = ""
        for k in range(len(evidence)):
          evidenceStore += evidence[k]['snippet']
        allEvi.append(evidenceStore)
    
    numSupporting = list(map(str, numSupporting))
    numRefuting = list(map(str, numRefuting))

    tmp = ""
    for val in numSupporting:
      tmp += val + ";"
    numSupporting = tmp

    tmp2 = ""
    for val in numRefuting:
      tmp2 += val + ";"
    numRefuting = tmp2

    tmp3 = ""
    for val in numSubQLabel:
      tmp3 += val + ";"
    numSubQLabel = tmp3

    tmp4 = ""
    for val in allEvi:
      tmp4 += val + ";"
    allEvi = tmp4

    print(numSupporting)
    print(totalSupporting)
    print(numRefuting)
    print(totalRefuting)

    df.loc[i, 'decomp-total-support'] = totalSupporting
    df.loc[i, 'decomp-total-refute'] = totalRefuting
    df.loc[i, 'decomp-supporting'] = numSupporting
    df.loc[i, 'decomp-refuting'] = numRefuting
    df.loc[i, 'decomp-label'] = numSubQLabel
    df.loc[i, 'decomp-evidence'] = allEvi

# print the response content
asyncio.run(evaluateV2(dfTest))

  0%|          | 1/200 [00:02<08:15,  2.49s/it]

0;2;0;
2
0;0;1;
1



  1%|          | 2/200 [00:05<09:04,  2.75s/it]

1;6;0;5;
12
0;0;4;2;
6



  2%|▏         | 3/200 [00:06<07:01,  2.14s/it]

...

1;6;2;
9
0;4;4;
8



100%|█████████▉| 199/200 [1:25:11<00:31, 31.24s/it]

4;0;0;
4
2;6;8;
16



100%|██████████| 200/200 [1:25:39<00:00, 25.70s/it]

1;0;1;
2
6;1;1;
8


In [ ]:
dfTest.head()

In [ ]:
dfTest.to_csv("./result/evalDecompDirectTest-stage3.csv", index=False, encoding = 'utf-8-sig', header=True, escapechar='',)

## Compute fact-checking performance

## Note

Please run this section with results from Method 1 or 2 to evaluate their accuracy scores. Do not run Methods 1 and 2 sequentially before running this section as it may override the results from either one of the methods.

### We compute the overall veracity score of the claim and award it a label

Final verarcity score will be based on 3 categories, True/False/UnableToEval represented by 1/0/-1 for easier evaluation, similar to the annotated dataset

**Mostly True labels are awarded if:**

Total supporting >= 50% of total evidence found

**Mostly False labels are awarded if:**

Total supporting < 50% of total evidence found


**Unable to Eval labels are awarded if:**

Total evidence = 0


**Quin performance** col will reflect 1 if quin's result matches that of the annotator's and 0 otherwise


In [ ]:
def evalDecomp(df):
  Accuracy= 0
  Unmatched = 0
  Matched = 0
  UnableToEval= 0
  
  for i in tqdm(range(len(df))):
    support = df.loc[i, 'decomp-total-support']
    refute = df.loc[i, 'decomp-total-refute']
    total = support + refute
    if total > 0:
      if round((support/total)*100) >= 50:
        df.loc[i, 'decomp-label-binary'] = 1
      else:
        df.loc[i, 'decomp-label-binary'] = 0
    else:
      df.loc[i, 'decomp-label-binary'] = -1
      UnableToEval += 1

  for i in tqdm(range(len(df))):
    if df.loc[i, 'label-binary'] == df.loc[i, 'decomp-label-binary']:
      df.loc[i, 'decomp-performance'] = 1
      Matched += 1
    else:
      df.loc[i, 'decomp-performance'] = 0
      Unmatched += 1


  Accuracy = round((Matched / (Unmatched + Matched)) * 100)
  print("\nAccuracy: {}%".format(Accuracy))
  print("Matched: {}".format(Matched))
  print("Unmatched: {}".format(Unmatched))
  print("UnableToEval: {}".format(UnableToEval))

evalDecomp(dfTest)


100%|██████████| 200/200 [00:00<00:00, 10030.98it/s]


Accuracy: 53%
Matched: 106
Unmatched: 94
UnableToEval: 7


In [ ]:
dfTest.to_csv("./result/evalDecompDirectTest-stage4.csv", index=False, encoding = 'utf-8-sig', header=True, )